<a href="https://colab.research.google.com/github/Evereth87/ML-Open-Source-Implementations/blob/master/Copy_of_original_SGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/Evereth87/Original.git

Cloning into 'Original'...
remote: Enumerating objects: 5528, done.
remote: Counting objects: 100% (5528/5528), done.
remote: Compressing objects: 100% (5484/5484), done.
remote: Total 5528 (delta 41), reused 5521 (delta 40), pack-reused 0
Receiving objects: 100% (5528/5528), 184.05 MiB | 37.31 MiB/s, done.
Resolving deltas: 100% (41/41), done.
Checking out files: 100% (5502/5502), done.


In [0]:
pip install Keras==2.1.6

     |████████████████████████████████| 348kB 6.9MB/s 
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [0]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import SGD
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *



Using TensorFlow backend.


In [0]:
train_path = 'Original/train'
valid_path = 'Original/valid'
test_path = 'Original/test' 



In [0]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=['cezanne', 'degas', 'gaugain', 'monet', 'raffaello', 'rembrandt', 'renoir', 'rubens', 'tiziano', 'vangogh'], batch_size=32)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size=(224,224), classes=['cezanne', 'degas', 'gaugain', 'monet', 'raffaello', 'rembrandt', 'renoir', 'rubens', 'tiziano', 'vangogh'], batch_size=32)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=['cezanne', 'degas', 'gaugain', 'monet', 'raffaello', 'rembrandt', 'renoir', 'rubens', 'tiziano', 'vangogh'], batch_size=32)

Found 3997 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 500 images belonging to 10 classes.


In [0]:
RaphaelModel = keras.applications.vgg16.VGG16()

W0723 15:27:06.473481 140553635501952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:68: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0723 15:27:06.521452 140553635501952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:508: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0723 15:27:06.529714 140553635501952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3837: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0723 15:27:06.566831 140553635501952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3661: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



553467904/553467096 [==============================] - 20s 0us/step


W0723 15:27:28.287972 140553635501952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:168: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0723 15:27:28.289469 140553635501952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:175: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [0]:
RaphaelModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
model = Sequential()
for layer in RaphaelModel.layers[:-1]: 
	model.add(layer)

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
for layer in model.layers:
	layer.trainable = False  
  
model.add(Dense(10, activation='softmax'))  

model.summary()  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
model.compile(SGD(lr=0.001), loss='categorical_crossentropy', metrics=['acc'])

stop = EarlyStopping(monitor='val_loss', verbose = 1, mode='min', patience=6, )

history = model.fit_generator(train_batches, steps_per_epoch=125,
					validation_data=valid_batches, validation_steps=32, epochs=50, verbose=2, callbacks=[stop])




W0723 15:27:32.523447 140553635501952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:757: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0723 15:27:32.619569 140553635501952 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


 - 33s - loss: 2.2204 - acc: 0.2869 - val_loss: 1.8383 - val_acc: 0.3800
Epoch 2/50
 - 24s - loss: 1.5602 - acc: 0.4670 - val_loss: 1.6012 - val_acc: 0.4590
Epoch 3/50
 - 25s - loss: 1.3259 - acc: 0.5429 - val_loss: 1.4788 - val_acc: 0.5000
Epoch 4/50
 - 24s - loss: 1.1880 - acc: 0.5960 - val_loss: 1.3959 - val_acc: 0.5260
Epoch 5/50
 - 24s - loss: 1.0965 - acc: 0.6339 - val_loss: 1.3653 - val_acc: 0.5480
Epoch 6/50
 - 24s - loss: 1.0249 - acc: 0.6539 - val_loss: 1.3536 - val_acc: 0.5640
Epoch 7/50
 - 25s - loss: 0.9660 - acc: 0.6728 - val_loss: 1.3542 - val_acc: 0.5620
Epoch 8/50
 - 24s - loss: 0.9157 - acc: 0.6950 - val_loss: 1.3076 - val_acc: 0.5720
Epoch 9/50
 - 24s - loss: 0.8706 - acc: 0.7153 - val_loss: 1.2927 - val_acc: 0.5860
Epoch 10/50
 - 24s - loss: 0.8330 - acc: 0.7256 - val_loss: 1.2758 - val_acc: 0.5740
Epoch 11/50
 - 24s - loss: 0.8023 - acc: 0.7390 - val_loss: 1.2815 - val_acc: 0.5890
Epoch 12/50
 - 24s - loss: 0.7703 - acc: 0.7531 - val_loss: 1.2721 - val_acc: 0.5890


In [0]:
#model.save(Original3/Models/ProjectModel3.h5)